# Imports

In [1]:
import win32com.client
import numpy as np
import time
import os
from IPython.display import display, Javascript
from IPython import get_ipython

# OLE Python-Simrna
This section was designed to perform tests and understand how to connect Python with Simnra. Each code cell does one thing, and if all cells are executed, it should perform right. For obvious reasons, there is a need to design a script which generates multiple spectra automatically, since this ipynb file needs to be run cell by cell. To execute it automatically, check the last Section

## Simnra.App

### App initialization

In [ ]:
application = win32com.client.Dispatch('Simnra.App')
#application.Reset()
application.Show()

In [ ]:
print('Simrna version: '+application.Version)

## Simnra.Setup

### Setup initialization

In [ ]:
setup = win32com.client.Dispatch('Simnra.Setup')

### Setup settings

In [ ]:
setup.Alpha = np.random.uniform(0,5) #incident angle
time.sleep(0.5)
setup.Energy = np.random.uniform(1400,2000) #beam energy
time.sleep(0.5)
setup.Theta = np.random.uniform(135,165) #exit angle

In [ ]:
print('Incident angle \u03B1 (deg): '+str(setup.Alpha))
print('Energy (keV): '+str(setup.Energy))
print('Scattering angle \u03B8 (deg): '+str(setup.Theta))

## Simnra.Target

### Target initialization


In [ ]:
target = win32com.client.Dispatch('Simnra.Target')

### Target settings

In [ ]:
listofelements= ["Ge","Si","Au"] #to be completed
element = np.random.choice(listofelements)
thick = np.random.uniform(100,500)
print('Element: ' +str(element))
print('Thickness: '+str(thick))

In [ ]:
#ensure that the target only has 1 layer
target.DeleteAllLayer()
time.sleep(0.25)
for i in range(target.TotalNumberOfElements):
    target.DeleteElement(1,i)
    time.sleep(0.25)
target.AddLayer()
time.sleep(0.5)
target.SetLayerThickness(1,thick) #set layer thickness
time.sleep(0.5)
target.AddElement(1) #add element (element has to be defined)
time.sleep(0.5)
target.SetElementName(1,1,element) #set element name
time.sleep(0.5)
target.SetElementConcentration(1,1,1) #set element concentration

In [ ]:
print('Number of layers: '+ str(target.NumberOfLayers))
print('Number of elements: '+ str(target.TotalNumberOfElements))
print('Elements present: '+str([target.ElementName(1,i+1) for i in range(target.TotalNumberOfElements)]))
print('Concentration of elements: '+str([(target.ElementName(1,i+1),target.GetElementConcentration(1,i+1)) for i in range(target.TotalNumberOfElements)]))
print('Target thickness: '+str(target.Thickness))

### Target reset (if needed)

In [ ]:
target.DeleteAllLayer()
for i in range(target.TotalNumberOfElements):
    target.DeleteElement(1,i)
target.AddLayer()

## Simnra.CrossSec

### Cross Section initialization

In [ ]:
crosssection = win32com.client.Dispatch('Simnra.CrossSec')

### Cross Section settings

In [ ]:
crosssection.SelectRutherfordAll()
crosssection.SetEmin(1,0.001)
crosssection.SetEMax(1,4999.999)

In [ ]:
print('CrossSection choosen: '+ str(crosssection.ReactionsChoosen))
print('Number of available cross-section data sets: '+ str(crosssection.Count))

## Simnra.Spectrum

### Spectrum initialization

In [ ]:
spectrum = win32com.client.Dispatch('Simnra.Spectrum')

### Spectrum settings

## Calculate Spectra

In [ ]:
application.CalculateSpectrum()

## Data exporting

In [ ]:
spectra_data=[]
labels_data=[]

In [ ]:
data=spectrum.GetDataArray(2)
data=list(data)
spectra_data.append(data)
aux=[]
aux=[setup.Alpha,setup.Energy,setup.Theta,target.Thickness,[list((target.ElementName(1,i+1),target.GetElementConcentration(1,i+1))) for i in range(target.TotalNumberOfElements)]]
labels_data.append(aux)

In [ ]:
data = 'spectra_data.txt'
labels = 'labels_data.txt'
with open(data, 'w') as file:
    list_as_string = '\n'.join(map(str, spectra_data))
    file.write(list_as_string)
with open(labels, 'w') as file:
    list_as_string = '\n'.join(map(str, labels_data))
    file.write(list_as_string)

## Reset objects

This is just an idea of how it might be possible to have only one instance of the SIMNRA running for all the spectra generation instead of one instance of the SIMNRA for each spectra generation

In [ ]:
setup,target,spectrum,crosssection = None,None,None,None

# Automatically Spectra Generation
**Note:** don't forget to run the cell that contains all the imports necessary in order to run the code, located in the first Section "Imports".

Generate .txt files where the data will be stored and initialize lists for further use.

**Caution:** running this will overwrite the data.txt and labels.txt files, which can lead into losing all the spectra data!

In [19]:
f= open('labels.txt', 'w')
f= open('data.txt', 'w')

## Code

In [20]:
def calculate_spectra(iterations,element,thickness,alpha,energy,theta):

    for _ in range(iterations):
        #start simnra

        application = win32com.client.Dispatch('Simnra.App')
        application.Show()

        #################################################################

        #start setup

        time.sleep(2)
        setup = win32com.client.Dispatch('Simnra.Setup')

        #setup settings

        time.sleep(0.25)
        setup.Alpha = alpha #incident angle

        time.sleep(0.25)
        setup.Energy = energy #beam energy

        time.sleep(0.25)
        setup.Theta = theta #exit angle


        print('Incident angle \u03B1 (deg): '+str(setup.Alpha))
        print('Energy (keV): '+str(setup.Energy))
        print('Scattering angle \u03B8 (deg): '+str(setup.Theta))
        print()

        #################################################################

        #start target

        target = win32com.client.Dispatch('Simnra.Target')

        #target settings

        #ensure that the target only has 1 layer

        target.DeleteAllLayer()
        time.sleep(0.25)
        for i in range(target.TotalNumberOfElements):
            target.DeleteElement(1,i)
            time.sleep(0.25)
        target.AddLayer()
        time.sleep(0.5)

        #layer settings

        target.SetLayerThickness(1,thickness) #set layer thickness
        time.sleep(0.4)
        target.AddElement(1) #add empty element
        time.sleep(0.4)
        target.SetElementName(1,1,element) #set element name
        time.sleep(0.4)
        target.SetElementConcentration(1,1,1) #set element concentration


        print('Number of layers: '+ str(target.NumberOfLayers))
        print('Number of elements: '+ str(target.TotalNumberOfElements))
        print('Elements present: '+str([target.ElementName(1,i+1) for i in range(target.TotalNumberOfElements)]))
        print('Concentration of elements: '+str([(target.ElementName(1,i+1),target.GetElementConcentration(1,i+1)) for i in range(target.TotalNumberOfElements)]))
        print('Target thickness: '+str(target.Thickness))
        print()

        #################################################################

        #start cross section

        crosssection = win32com.client.Dispatch('Simnra.CrossSec')

        #cross section settings

        time.sleep(0.5)
        crosssection.SelectRutherfordAll()

        time.sleep(0.5)
        crosssection.SetEmin(1,0.001)
        crosssection.SetEMax(1,4999.999)


        print('CrossSection choosen: '+ str(crosssection.ReactionsChoosen))
        print('Number of available cross-section data sets: '+ str(crosssection.Count))

        #################################################################

        #start spectrum

        spectrum = win32com.client.Dispatch('Simnra.Spectrum')

        #calculate spectrum

        application.CalculateSpectrum()


        #data exporting

        datalist=list(spectrum.GetDataArray(2))
        labelslist=[setup.Alpha,setup.Energy,setup.Theta,target.Thickness,[list((target.ElementName(1,i+1),target.GetElementConcentration(1,i+1))) for i in range(target.TotalNumberOfElements)]]


        with open('data.txt', 'a') as file:
            list_as_string = ' '.join(map(str, datalist))
            file.write(list_as_string)
            file.write('\n')
        with open('labels.txt', 'a') as file:
            list_as_string = ' '.join(map(str, labelslist))
            file.write(list_as_string)
            file.write('\n')

        thickness+=50
        if thickness>500:
            break
        print()
        print()
        print()
        print()
        print()

In [6]:
alpha = np.random.uniform(0,5) #incident angle
energy = np.random.uniform(1400,2000) #beam energy
theta = np.random.uniform(135,165) #exit angle
listofelements= ["Ge","Si","Au"] #list of elements
element = np.random.choice(listofelements) #pick an element
thickness = np.random.uniform(100,500) #layer thickness

In [21]:
calculate_spectra(iterations=10,
                  element='Ge',
                  thickness=100,
                  alpha=2,
                  energy=1800,
                  theta=150)

Incident angle α (deg): 2.0
Energy (keV): 1800.0
Scattering angle θ (deg): 150.0

Number of layers: 1
Number of elements: 1
Elements present: ['Ge']
Concentration of elements: [('Ge', 1.0)]
Target thickness: 100.0

CrossSection choosen: True
Number of available cross-section data sets: 16





Incident angle α (deg): 2.0
Energy (keV): 1800.0
Scattering angle θ (deg): 150.0

Number of layers: 1
Number of elements: 1
Elements present: ['Ge']
Concentration of elements: [('Ge', 1.0)]
Target thickness: 150.0

CrossSection choosen: True
Number of available cross-section data sets: 16





Incident angle α (deg): 2.0
Energy (keV): 1800.0
Scattering angle θ (deg): 150.0

Number of layers: 1
Number of elements: 1
Elements present: ['Ge']
Concentration of elements: [('Ge', 1.0)]
Target thickness: 200.0

CrossSection choosen: True
Number of available cross-section data sets: 16





Incident angle α (deg): 2.0
Energy (keV): 1800.0
Scattering angle θ (deg): 150.0

Number of layers: 1
Number of ele

In [ ]:
calculate_spectra(iterations=5,
                  element=element,
                  thickness=thickness,
                  alpha=alpha,
                  energy=energy,
                  theta=theta)

In [14]:
with open('data.txt', 'r') as file:
    input_string = file.read()
lines = input_string.split('\n')
x = []

for line in lines:
    values = line.split()  # Split line into individual values (space-separated)
    int_values = [float(value) for value in values]  # Convert string values to list of single integers
    x.append(int_values)

del x[len(x)-1]
print(x)
y=[200, 200]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [6]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=1,random_state=2)